In [2]:
import mediapipe as mp
import pickle as pkl
import os
import numpy as np
import cv2
import clip
import torch
from PIL import Image
import pandas
import collections

from utilz import *


/home/user2_4/anaconda3/envs/tf_37/lib/python3.7/site-packages/clip/clip.py:24: UserWarning: PyTorch version 1.7.1 or higher is recommended
  warnings.warn("PyTorch version 1.7.1 or higher is recommended")


In [ ]:
ids,paths,nlps,classes,modes=load_data(r'/home/user2_4/code/shujuji_jupyter/fold0_train.csv')

# mediapipe - 提取所有视频特征（只运行一次）

In [3]:
# ============ mediapipe: 提取所有视频特征（只运行一次）============
MODE = 'face'  # 可选 'ori' 或 'face'
max_len = 16

# 加载数据
ids, paths, nlps, classes, modes = load_data(r'/home/user2_4/code/shujuji_jupyter/fold0_train.csv')

visual_all = {}  # 用路径作为key

for path, classe, mode in zip(paths, classes, modes):
    print(path)
    if path in visual_all:  # 如果已经处理过，跳过
        continue
        
    if MODE == 'ori':
        cap = cv2.VideoCapture(path)
        images = []
        while cap.isOpened():
            success, image = cap.read()
            if success:
                images.append(cv2.resize(image, (224, 224)))
            else:
                break
        cap.release()
        
        if len(images) == 0:
            images = np.zeros((max_len, 224, 224, 3))
        else:
            images = np.array(images)
            if len(images) > max_len:
                images = images[::int(len(images)/max_len)]
            if len(images) < max_len:
                images = np.concatenate([images, np.zeros((max_len-len(images), 224, 224, 3))], axis=0)
        
        visual_all[path] = images[:max_len]
        
    elif MODE == 'face':
        mp_face_detection = mp.solutions.face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)
        cap = cv2.VideoCapture(path)
        faces = []
        while cap.isOpened():
            success, image = cap.read()
            if success:
                img_h, img_w, img_c = image.shape
                image.flags.writeable = False
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = mp_face_detection.process(image_rgb)
                # 保持RGB格式，不转灰度图
                if results.detections:
                    for detection in results.detections:
                        xmin = int(detection.location_data.relative_bounding_box.xmin * img_w)
                        ymin = int(detection.location_data.relative_bounding_box.ymin * img_h)
                        width = int(detection.location_data.relative_bounding_box.width * img_w)
                        height = int(detection.location_data.relative_bounding_box.height * img_h)
                        ymin = max(0, ymin)
                        xmin = max(0, xmin)
                        # 使用RGB图像裁剪人脸，保持3通道
                        face = cv2.resize(image_rgb[ymin:ymin+height, xmin:xmin+width], (224, 224))
                        faces.append(face)
            else:
                break
        cap.release()
        
        if len(faces) == 0:
            faces = np.zeros((max_len, 224, 224, 3))  # 修正：添加通道维度
        else:
            faces = np.array(faces)
            if len(faces) > max_len:
                faces = faces[::int(len(faces)/max_len)]
            if len(faces) < max_len:
                faces = np.concatenate([faces, np.zeros((max_len-len(faces), 224, 224, 3))], axis=0)  # 修正：添加通道维度
        
        visual_all[path] = faces[:max_len]

# 保存所有特征
save_features(visual_all, './data2025/visual_face_all.pkl')
print(f"共提取 {len(visual_all)} 个视频的特征")

/home/user2_4/code/shujuji_jupyter/shipin2/pz_gxq1.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/lar_gxq4.mp4


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


/home/user2_4/code/shujuji_jupyter/shipin2/lbj_gxq2.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/czj_gxq1.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/wd_gxq3.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/ghx_gxq2.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/lky_gxq4.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/qyc_gxq1.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/lar_gxq3.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/yrh_gxq1.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/czj_gxq2.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/lky_gxq2.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/ghx_gxq1.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/wdn_kl2.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/fq_kl1.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/lzl_kl2.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/lky_kl1.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/wd_kl4.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/wd_kl2.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/czj_kl1.mp4
/ho

# OpenFace

In [ ]:
for path,classe,mode in zip (paths,classes,modes):

    os.system(r'D://OpenFace-OpenFace_2.2.0//x64//Release//FaceLandmarkVidMulti -f {} -out_dir ./data/OFprocessed/{}/'.format(path,path.split(':\\')[-1]))


In [ ]:
for path,classe,mode in zip (paths,classes,modes):

    os.system(r'D://OpenFace-OpenFace_2.2.0//x64//Release//FeatureExtraction.exe -f {} -out_dir ./data/OFprocessed1/{}/ - pose'.format(path,path.split(':\\')[-1]))


KeyboardInterrupt: 

In [ ]:
FAU_list=OP_para()

features_path='./data/OFprocessed'
FAU_features={'train':[],'test':[]}
max_len=30
# file = []
for path,classe,mode in zip (paths,classes,modes):
    root_path='./data/OFprocessed'
    features_path = ''
#   原来  file=features_path+path.split(':\\')[-1][-4:]+'.csv'
    str_list = path.split('\\')
    for i in range(1,len(str_list)):
        features_path +=  r'/' +str_list[i]
    features_path=   root_path +   features_path +    r'/'+ str_list[-1].split('.')[0] + '.csv'
    
#     file.append(features_path)
    features = []

    try:
        file_ct=pandas.read_csv(features_path)
        #删除列名前置尾随的空格
        file_ct.columns = file_ct.columns.str.strip()
        #捉取出现最多的人脸
        dominant_face=list(collections.Counter(list(file_ct['face_id'])).keys())[0]


        for i in range(len(file_ct['frame'])):
            if file_ct['face_id'][i]==dominant_face and file_ct['success'][i]==1:
                fau_ft=[]
                for fau in FAU_list:
                    fau_ft.append(file_ct[fau][i])
            #print('fau_ft shape:', np.array(fau_ft).shape)
            features.append(fau_ft)
        #print(features)
    except Exception as e:
        print('Error:', e)
        print('no face',features_path)
        #print('np.zeros((max_len,49)) shape:', np.zeros((max_len,49)).shape)
        features=np.zeros((max_len,709))


    if len(features)>max_len:
        features=features[::int(len(features)/max_len)]
    if len(features)<max_len:
        print("Features shape:", np.array(features).shape)
        features=np.concatenate([features,np.zeros((max_len-len(features),709))],axis=0)

    FAU_features[mode].append(features[:max_len])

save_features(FAU_features,'./data/visual_OFfts.pkl')

Features shape: (18, 709)


# CLIP - 提取所有视频特征（只运行一次）

In [ ]:
# ============ CLIP: 提取所有视频特征（只运行一次）============
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, preprocess = clip.load('ViT-B/32', device=device)

max_len = 16

# 加载数据
ids, paths, nlps, classes, modes = load_data(r'/home/user2_4/code/shujuji_jupyter/fold0_train.csv')

visual_all = {}  # 用路径作为key

with torch.no_grad():
    for path, classe, mode in zip(paths, classes, modes):
        print(path)
        if path in visual_all:  # 如果已经处理过，跳过
            continue
            
        cap = cv2.VideoCapture(path)
        image_features = []
        while cap.isOpened():
            success, image = cap.read()
            if success:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image_ = preprocess(Image.fromarray(np.uint8(image))).unsqueeze(0).to(device)
                image_ft = model.encode_image(image_).cpu().detach().numpy()[0]
                image_features.append(image_ft)
            else:
                break
        cap.release()
        
        if len(image_features) == 0:
            image_features = np.zeros((max_len, 512))
        else:
            image_features = np.array(image_features)
            if len(image_features) > max_len:
                image_features = image_features[::int(len(image_features)/max_len)]
            if len(image_features) < max_len:
                image_features = np.concatenate([image_features, np.zeros((max_len-len(image_features), 512))], axis=0)
        
        visual_all[path] = image_features[:max_len]

# 保存所有特征
save_features(visual_all, './data/visual_clip_all.pkl')
print(f"共提取 {len(visual_all)} 个视频的特征")

100%|███████████████████████████████████████| 338M/338M [00:30<00:00, 11.6MiB/s]


/home/user2_4/code/shujuji_jupyter/shipin2/pz_gxq1.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/lar_gxq4.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/lbj_gxq2.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/czj_gxq1.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/wd_gxq3.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/ghx_gxq2.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/lky_gxq4.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/qyc_gxq1.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/lar_gxq3.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/yrh_gxq1.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/czj_gxq2.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/lky_gxq2.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/ghx_gxq1.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/wdn_kl2.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/fq_kl1.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/lzl_kl2.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/lky_kl1.mp4
/home/user2_4/code/shujuji_jupyter/shipin2/wd_kl4.mp4
/

# 根据fold划分特征（每个fold运行）

In [4]:
# ============ 根据fold划分特征（每个fold运行）============
import pickle

def load_features(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

# 选择要划分的特征类型
feature_type = 'mp'  # 可选 'clip' 或 'mp'

if feature_type == 'clip':
    visual_all = load_features('./data/visual_clip_all.pkl')
    output_prefix = 'visual_clip'
else:
    visual_all = load_features('./data2025/visual_face_all.pkl')
    output_prefix = 'visual_face'

# 对每个fold进行划分
for fold in [0, 1, 2]:
    ids, paths, nlps, classes, modes = load_data(f'/home/user2_4/code/shujuji_jupyter/fold{fold}_train.csv')
    
    visual = {'train': [], 'test': []}
    
    for path, classe, mode in zip(paths, classes, modes):
        if path in visual_all:
            visual[mode].append(visual_all[path])
        else:
            print(f"警告: 找不到视频特征 {path}")
    
    # 保存当前fold的特征
    save_features(visual, f'./data2025/fold{fold}_{output_prefix}.pkl')
    print(f"Fold {fold}: train={len(visual['train'])}, test={len(visual['test'])}")

Fold 0: train=355, test=167
Fold 1: train=346, test=176
Fold 2: train=343, test=179
